### Import Modules

In [1]:
import fiona
from shapely.geometry import shape, Point
from shapely import speedups
speedups.enable()
import datetime
import pandas

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Set paths

In [2]:
points = "/users/danielcorcoran/desktop/test_coordinates_10K.csv"
shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"

### Import data and preview

In [3]:
#data = pandas.read_csv("/users/danielcorcoran/desktop/test_coordinates_10K.csv")

In [4]:
#Create test data

import numpy
numpy.random.seed(0)

x  = [numpy.random.uniform(140,150) for n in range(50000)]
y  = [numpy.random.uniform(-32,-40) for n in range(50000)]
data = pandas.concat([pandas.Series(x), pandas.Series(y)], axis = 1)
data.columns = ['longitude', 'latitude']

In [5]:
data.head()

,longitude,latitude
0,145.488135,-34.461163
1,147.151894,-32.496042
2,146.027634,-39.910281
3,145.448832,-37.752722
4,144.236548,-38.060557


In [6]:
print('Checking data shape:\n{}'.format(data.shape))

Checking data shape:
(50000, 2)


In [7]:
print('Checking data headers:\n{}'.format(list(data.columns)))

Checking data headers:
['longitude', 'latitude']


In [8]:
long_name = 'longitude'
lat_name = 'latitude'

### Create list of polygons containing Victoria only from LGA shapefile

In [9]:
all_polygons = [polygon for polygon in fiona.open(shapefile)]

In [10]:
vic_polygons = []

for polygon in all_polygons:
    if polygon['properties']['STE_NAME16'] == 'Victoria' and polygon['geometry'] is not None:
        vic_polygons.append(polygon)
        
print(len(vic_polygons))

80


### Build spatial tree

In [11]:
from rtree import index
rtree_index = index.Index()

In [12]:
simplified_polygons = []
tolerance = 0.00005

for position, polygon in enumerate(vic_polygons):
    rtree_index.insert(position, shape(polygon['geometry']).bounds)
    shapeobject = shape(polygon['geometry'])
    simplified_polygons.append(shapeobject.simplify(tolerance = tolerance))
    

### Process  (with apply)

In [13]:
# Create function to return the properties for a shapefile if a match is found in rtree_index
def return_properties(point):
    for index in rtree_index.intersection(point.coords[0]):
        #if point.within(shape(vic_polygons[index]['geometry'])):
        if point.within(simplified_polygons[index]):
            return vic_polygons[index]['properties']

In [14]:
# Create new features with point objects using long and lat features
data['point'] = (list(zip(data[long_name], data[lat_name])))
data["shapely_point"] = data['point'].apply(Point)

In [15]:
from multiprocessing import Pool
p = Pool(4)

In [16]:
%%time

data['results'] = list(p.map(return_properties, data["shapely_point"]))

# Join original data with results series split into a new dataframe
# (one column for each property)
data = pandas.concat([data, data['results'].apply(pandas.Series)], axis = 1)

# Drop irrelevant columns
data.drop(['shapely_point', 'results', 'point'], axis = 1, inplace = True)

CPU times: user 14.4 s, sys: 309 ms, total: 14.8 s
Wall time: 26.7 s


In [17]:
print(data.head())

    longitude   latitude LGA_CODE16         LGA_NAME16 STE_CODE16 STE_NAME16  \
0  145.488135 -34.461163        NaN                NaN        NaN        NaN   
1  147.151894 -32.496042        NaN                NaN        NaN        NaN   
2  146.027634 -39.910281        NaN                NaN        NaN        NaN   
3  145.448832 -37.752722      27450   Yarra Ranges (S)          2   Victoria   
4  144.236548 -38.060557      22490  Golden Plains (S)          2   Victoria   

   AREASQKM16  
0         NaN  
1         NaN  
2         NaN  
3   2468.2050  
4   2703.3898  


In [18]:
print('Points found {}. Total Points {}. % Points found {:.2%}'.format(data['LGA_CODE16'].notnull().sum(),
                                                                        data['LGA_CODE16'].shape[0],
                                (data['LGA_CODE16'].notnull().sum()/data['LGA_CODE16'].shape[0])))

Points found 14280. Total Points 50000. % Points found 28.56%


### Export

In [19]:
data.to_csv('/users/danielcorcoran/desktop/test_points.csv', index = False)